In [1]:
!pip install wandb --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 3.4 MB/s eta 0:00:00


In [2]:
!pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchsummary import summary
import torchvision 
import os
import gc
from tqdm import tqdm
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import glob
import wandb
import math
import matplotlib.pyplot as plt
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", DEVICE)

Device:  cuda


In [4]:
from google.colab import drive # Link your drive if you are a colab user
drive.mount('/content/drive') 

Mounted at /content/drive


In [5]:
config = {
    'batch_size': 64, 
    'lr': 0.001,
    'epochs': 230,
    'loss_weight':0.2,
}

In [6]:
# TODO: Use the same Kaggle code from HW1P2
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"adritadas","key":"4d8e5e73b5126973227f2e938ba1249f"}') 
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 KB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73272 sha256=1d1fe1cfb0373134388942827dba8383090f920f88234bbad5d8f7745b541e85
  Stored in directory: /root/.cache/pip/wheels/d4/02/ef/3f8c8d86b8d5388a1d3155876837f1a1a3143ab3fc2ff1ffad
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.13
    Uninstalling kaggle-1.5.13:
      Successfully uninstalled kaggle-1.5.13


In [7]:
!mkdir '/content/data'

!kaggle competitions download -c 11-785-s23-hw2p2-classification
!unzip -qo '11-785-s23-hw2p2-classification.zip' -d '/content/data'

!kaggle competitions download -c 11-785-s23-hw2p2-verification
!unzip -qo '11-785-s23-hw2p2-verification.zip' -d '/content/data'

100% 1.71G/1.72G [00:53<00:00, 32.9MB/s]
100% 1.72G/1.72G [00:53<00:00, 34.3MB/s]
 83% 14.0M/16.8M [00:00<00:00, 22.0MB/s]
100% 16.8M/16.8M [00:01<00:00, 16.6MB/s]


In [8]:
DATA_DIR    = '/content/data/11-785-s23-hw2p2-classification'
TRAIN_DIR   = os.path.join(DATA_DIR, "train") 
VAL_DIR     = os.path.join(DATA_DIR, "dev")
TEST_DIR    = os.path.join(DATA_DIR, "test")

# Transforms using torchvision - Refer https://pytorch.org/vision/stable/transforms.html

train_transforms = torchvision.transforms.Compose([
                torchvision.transforms.RandomHorizontalFlip(0.5),
                torchvision.transforms.ColorJitter(brightness = 0.2, contrast = 0.2, saturation = 0.2),
                torchvision.transforms.RandomGrayscale(p=0.2),
                torchvision.transforms.RandAugment(),
                torchvision.transforms.ToTensor(),
                
                ])
# Most torchvision transforms are done on PIL images. So you convert it into a tensor at the end with ToTensor()
# But there are some transforms which are performed after ToTensor() : e.g - Normalization
# Normalization Tip - Do not blindly use normalization that is not suitable for this dataset

valid_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])


train_dataset   = torchvision.datasets.ImageFolder(TRAIN_DIR, transform= train_transforms)
valid_dataset   = torchvision.datasets.ImageFolder(VAL_DIR, transform= valid_transforms)
# You should NOT have data augmentation on the validation set. Why?


# Create data loaders
train_loader = torch.utils.data.DataLoader(
    dataset     = train_dataset, 
    batch_size  = config['batch_size'], 
    shuffle     = True,
    num_workers = 4, 
    pin_memory  = True
)

valid_loader = torch.utils.data.DataLoader(
    dataset     = valid_dataset, 
    batch_size  = config['batch_size'],
    shuffle     = False,
    num_workers = 2
)

In [9]:
class ClassificationTestDataset(torch.utils.data.Dataset):

    def __init__(self, data_dir, transforms):
        self.data_dir   = data_dir
        self.transforms = transforms

        # This one-liner basically generates a sorted list of full paths to each image in the test directory
        self.img_paths  = list(map(lambda fname: os.path.join(self.data_dir, fname), sorted(os.listdir(self.data_dir))))

    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        return self.transforms(Image.open(self.img_paths[idx]))

In [10]:
test_dataset = ClassificationTestDataset(TEST_DIR, transforms = valid_transforms) #Why are we using val_transforms for Test Data?
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = config['batch_size'], shuffle = False,
                         drop_last = False, num_workers = 2)

In [11]:
print("Number of classes    : ", len(train_dataset.classes))
print("No. of train images  : ", train_dataset.__len__())
print("Shape of image       : ", train_dataset[0][0].shape)
print("Batch size           : ", config['batch_size'])
print("Train batches        : ", train_loader.__len__())
print("Val batches          : ", valid_loader.__len__())

Number of classes    :  7000
No. of train images  :  140000
Shape of image       :  torch.Size([3, 224, 224])
Batch size           :  64
Train batches        :  2188
Val batches          :  547


In [13]:
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
Net = torchvision.models.convnext_tiny()
print(Net)

ConvNeXt(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
    )
    (1): Sequential(
      (0): CNBlock(
        (block): Sequential(
          (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (1): Permute()
          (2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=96, out_features=384, bias=True)
          (4): GELU(approximate='none')
          (5): Linear(in_features=384, out_features=96, bias=True)
          (6): Permute()
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): CNBlock(
        (block): Sequential(
          (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (1): Permute()
          (2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=

In [ ]:
class TinyConvNeXtBlock(nn.Module):
  def __init__(self, in_channels, out_channels, expansion_ratio, dropout):
    super().__init__()
    hidden_dim = in_channels * expansion_ratio



    self.conv_1 = nn.Conv2d(in_channels, in_channels, kernel_size=7, padding=3, groups=in_channels)

    self.bn_1 = nn.BatchNorm2d(in_channels)

    
    self.conv_2 = nn.Conv2d(in_channels, hidden_dim, kernel_size=1, stride=1, padding=0)


    self.gelu = nn.GELU()

    
    self.conv_3 = nn.Conv2d(hidden_dim, out_channels, kernel_size=1, stride=1, padding=0)
    

    self.stoch_depth = torchvision.ops.StochasticDepth(p=dropout, mode="row")

  def forward(self, x):
    out = self.conv_1(x)
    out = self.bn_1(out)
    out = self.conv_2(out)
    out = self.gelu(out)
    out = self.conv_3(out)
    x = x + self.stoch_depth(out)
    return x

In [ ]:
class Network(nn.Module):

  def __init__(self, num_classes=7000):
    super().__init__()
    self.num_classes = num_classes
    self.layers = []

    start_dropout = 0.0  # Initial dropout probability
    end_dropout = 0.1

  
  

    self.expnum_block = [[4, 96, 3],
                        [4, 192, 3], 
                         [4, 384, 9],    
                         [4, 768, 3]
                         ]



   

    
    stem_channels = self.expnum_block[0][1]
    tiny_stem = nn.Sequential(
        nn.Conv2d(in_channels = 3, out_channels = stem_channels, kernel_size=4, stride=4),
        nn.BatchNorm2d(num_features = stem_channels)) 

   

    self.layers = nn.ModuleList()
    start_dropout = 0.0  # Initial dropout probability
    end_dropout = 0.1

    for i, (expansion_ratio, in_channels, num_blocks) in enumerate(self.expnum_block):
     dropout_probs = torch.linspace(start_dropout, end_dropout, num_blocks)

     for j in range(num_blocks):
       self.layers.append(TinyConvNeXtBlock(in_channels, in_channels, expansion_ratio, dropout=dropout_probs[j].item()))
       start_dropout = end_dropout
       end_dropout += 0.0001
      
    if i < len(self.expnum_block) - 1:
     self.layers.append(nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.Conv2d(in_channels, self.expnum_block[i+1][1], kernel_size=2, stride=2)
        ))

    self.embeddings = nn.Sequential(
    nn.AdaptiveAvgPool2d(1),
    nn.BatchNorm2d(self.expnum_block[-1][1]),
    nn.Flatten(),
)

    self.classification = nn.Linear(self.expnum_block[-1][1], self.num_classes)


  def forward(self, x, return_feats=False):
    """
    What is return_feats? It essentially returns the second-to-last-layer
    features of a given image. It's a "feature encoding" of the input image,
    and you can use it for the verification task. You would use the outputs
    of the final classification layer for the classification task.

    You might also find that the classification outputs are sometimes better
    for verification too - try both.
    """
    out = self.layers(x)
    feats = self.embeddings(out)

    out = self.classification(feats)
    
    
    if return_feats:
      return feats
    else:
      return out 

In [19]:
model = Network()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model= torch.nn.DataParallel(model)
model.to(device)

DataParallel(
  (module): Network(
    (layers): ModuleList(
      (0): ConvNeXtBlock(
        (conv_1): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
        (bn_1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv_2): Conv2d(96, 384, kernel_size=(1, 1), stride=(1, 1))
        (gelu): GELU(approximate='none')
        (conv_3): Conv2d(384, 96, kernel_size=(1, 1), stride=(1, 1))
        (stoch_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): ConvNeXtBlock(
        (conv_1): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
        (bn_1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv_2): Conv2d(96, 384, kernel_size=(1, 1), stride=(1, 1))
        (gelu): GELU(approximate='none')
        (conv_3): Conv2d(384, 96, kernel_size=(1, 1), stride=(1, 1))
        (stoch_depth): StochasticDepth(p=0.05000000074505806, mode=row)
      )
   

In [ ]:
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
criterion = torch.nn.CrossEntropyLoss(label_smoothing=0.2)
optimizer = torch.optim.SGD(model.parameters(), lr=config['lr'], momentum=0.9, weight_decay=1e-4)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=4, factor=0.5,mode = 'max',verbose=True)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(len(train_loader) * config['epochs']))
scaler = torch.cuda.amp.GradScaler()


# TODO: Implement a scheduler (Optional but Highly Recommended)
# You can try ReduceLRonPlateau, StepLR, MultistepLR, CosineAnnealing, etc.
# Good news. We have FP16 (Mixed precision training) implemented for you
# It is useful only in the case of compatible GPUs such as T4/V100

In [ ]:
def train(model, dataloader, optimizer, criterion):
    
    model.train()

    # Progress Bar 
    batch_bar   = tqdm(total=len(dataloader), dynamic_ncols=True, leave=False, position=0, desc='Train', ncols=5) 

    num_correct = 0
    total_loss  = 0

    for i, (images, labels) in enumerate(dataloader):
        
        optimizer.zero_grad() # Zero gradients

        images, labels = images.to(DEVICE), labels.to(DEVICE)
        
        with torch.cuda.amp.autocast(): # This implements mixed precision. Thats it! 
            outputs = model(images)
            loss    = criterion(outputs, labels)

        # Update no. of correct predictions & loss as we iterate
        num_correct     += int((torch.argmax(outputs, axis=1) == labels).sum())
        total_loss      += float(loss.item())

        # tqdm lets you add some details so you can monitor training as you train.
        batch_bar.set_postfix(
            acc         = "{:.04f}%".format(100 * num_correct / (config['batch_size']*(i + 1))),
            loss        = "{:.04f}".format(float(total_loss / (i + 1))),
            num_correct = num_correct,
            lr          = "{:.04f}".format(float(optimizer.param_groups[0]['lr']))
        )
        
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() 
        scheduler.step()

        # TODO? Depending on your choice of scheduler,
        # You may want to call some schdulers inside the train function. What are these?
      
        batch_bar.update() # Update tqdm bar

    batch_bar.close() # You need this to close the tqdm bar

    acc         = 100 * num_correct / (config['batch_size']* len(dataloader))
    total_loss  = float(total_loss / len(dataloader))

    return acc, total_loss

In [ ]:
def validate(model, dataloader, criterion):
  
    model.eval()
    batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True, position=0, leave=False, desc='Val', ncols=5)

    num_correct = 0.0
    total_loss = 0.0

    for i, (images, labels) in enumerate(dataloader):
        
        # Move images to device
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        
        # Get model outputs
        with torch.inference_mode():
            outputs = model(images)
            loss = criterion(outputs, labels)

        num_correct += int((torch.argmax(outputs, axis=1) == labels).sum())
        total_loss += float(loss.item())

        batch_bar.set_postfix(
            acc="{:.04f}%".format(100 * num_correct / (config['batch_size']*(i + 1))),
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            num_correct=num_correct)

        batch_bar.update()
        
    batch_bar.close()
    acc = 100 * num_correct / (config['batch_size']* len(dataloader))
    total_loss = float(total_loss / len(dataloader))
    return acc, total_loss

In [ ]:
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
wandb.login(key="a0b5e594abd47863b3c9a38a1319b7dbecb14246")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# Create your wandb run
run = wandb.init(
     name = "SmallConvNeXt-lr-1e-3-sgd", ## Wandb creates random run names if you skip this field
     reinit = True, ### Allows reinitalizing runs when you re-run this cell
     #run_id = run-qasgkozm-history:v0, #Insert specific run id here if you want to resume a previous run
     #resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
     project = "hw2p2-ablations", ### Project should be created in your wandb account 
     config = config ### Wandb Config for your run
 )

wandb: Currently logged in as: adritad (iw). Use `wandb login --relogin` to force relogin


In [ ]:
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
best_valacc = 0.0

for epoch in range(e,config['epochs']):

    curr_lr = float(optimizer.param_groups[0]['lr'])

    train_acc, train_loss = train(model, train_loader, optimizer, criterion)
    
    print("\nEpoch {}/{}: \nTrain Acc {:.04f}%\t Train Loss {:.04f}\t Learning Rate {:.04f}".format(
        epoch + 1,
        config['epochs'],
        train_acc,
        train_loss,
        curr_lr))
    
    val_acc, val_loss = validate(model, valid_loader, criterion)
    
    print("Val Acc {:.04f}%\t Val Loss {:.04f}".format(val_acc, val_loss))

    wandb.log({"train_loss":train_loss, 'train_Acc': train_acc, 'validation_Acc':val_acc, 
               'validation_loss': val_loss, "learning_Rate": curr_lr})
    

    
    # If you are using a scheduler in your train function within your iteration loop, you may want to log
    # your learning rate differently 

    # #Save model in drive location if val_acc is better than best recorded val_acc
    if val_acc >= best_valacc:
      #path = os.path.join(root, model_directory, 'checkpoint' + '.pth')
      print("Saving model")
      torch.save({'model_state_dict':model.state_dict(),
                  'optimizer_state_dict':optimizer.state_dict(),
                  'scheduler_state_dict':scheduler.state_dict(),
                  'val_acc': val_acc, 
                  'epoch': epoch}, "/content/drive/MyDrive/checkpoint/chk_"+str(epoch)+".pth")
      best_valacc = val_acc
      wandb.save('checkpoint.pth')
      # You may find it interesting to exlplore Wandb Artifcats to version your models
run.finish()


Epoch 183/230: 
Train Acc 99.9386%	 Train Loss 2.3901	 Learning Rate 0.0010


Val Acc 91.2191%	 Val Loss 3.2150
Saving model



Epoch 184/230: 
Train Acc 99.9300%	 Train Loss 2.3906	 Learning Rate 0.0010


Val Acc 91.2420%	 Val Loss 3.2014
Saving model



Epoch 185/230: 
Train Acc 99.9350%	 Train Loss 2.3908	 Learning Rate 0.0010


...


Epoch 227/230: 
Train Acc 99.9400%	 Train Loss 2.3880	 Learning Rate 0.0009


Val Acc 91.2449%	 Val Loss 3.1893



Epoch 228/230: 
Train Acc 99.9414%	 Train Loss 2.3878	 Learning Rate 0.0009


Val Acc 91.2506%	 Val Loss 3.1881


Train:  89%|████████▊ | 1941/2188 [13:46<01:44,  2.35it/s, acc=99.9533%, loss=2.3884, lr=0.0009, num_correct=124166]

In [ ]:
checkpoint = torch.load("/content/drive/MyDrive/checkpoint/chk_"+str(226)+".pth")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
optimizer.param_groups[0]['lr'] = 0.001
e = checkpoint['epoch'] + 1

FileNotFoundError: ignored

In [ ]:
def test(model,dataloader):

  model.eval()
  batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True, position=0, leave=False, desc='Test')
  test_results = []
  
  for i, (images) in enumerate(dataloader):
      # TODO: Finish predicting on the test set.
      images = images.to(DEVICE)

      with torch.inference_mode():
        outputs = model(images)

      outputs = torch.argmax(outputs, axis=1).detach().cpu().numpy().tolist()
      test_results.extend(outputs)
      
      batch_bar.update()
      
  batch_bar.close()
  return test_results

In [ ]:
test_results = test(model, test_loader)
print(test_results)

[2210, 5459, 762, 2928, 2863, 3610, 3126, 2953, 1988, 5882, 5331, 1392, 3637, 2292, 6609, 6991, 2623, 3199, 6878, 801, 6978, 5166, 848, 182, 341, 5238, 1259, 6664, 3317, 1820, 2559, 4417, 1894, 538, 1668, 2507, 899, 745, 89, 4957, 1572, 6554, 6933, 1843, 1303, 465, 1412, 6480, 5289, 3531, 3718, 5924, 6913, 2456, 2642, 4481, 1272, 600, 4492, 5470, 3777, 6893, 5326, 2555, 5627, 3018, 5426, 2464, 5736, 147, 4357, 5770, 1126, 3768, 2009, 3926, 5506, 3684, 5766, 2264, 6174, 2114, 2089, 5012, 322, 363, 4988, 5579, 304, 3537, 3968, 1556, 1989, 409, 5268, 368, 5142, 6275, 4433, 1283, 5285, 5181, 662, 4872, 4167, 3106, 1326, 914, 1115, 1886, 1778, 3796, 4176, 3915, 6902, 1403, 5098, 1643, 3847, 178, 5431, 2977, 5440, 6008, 6504, 3282, 5596, 2019, 2457, 3598, 1316, 2, 2876, 4633, 11, 3271, 4931, 3072, 901, 5499, 6243, 5663, 2981, 5154, 3131, 3056, 2907, 823, 6686, 4039, 3254, 6175, 5109, 2237, 5849, 3116, 165, 5440, 6418, 1067, 3244, 4451, 5069, 1443, 1267, 4797, 5369, 3942, 2441, 5212, 2071, 44

In [ ]:
with open("classification_early_submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(test_dataset)):
        f.write("{},{}\n".format(str(i).zfill(5) + ".jpg", test_results[i]))

In [ ]:
# This obtains the list of known identities from the known folder
known_regex = "/content/data/11-785-s23-hw2p2-verification/known/*/*"
known_paths = [i.split('/')[-2] for i in sorted(glob.glob(known_regex))]

# Obtain a list of images from unknown folders
unknown_dev_regex = "/content/data/11-785-s23-hw2p2-verification/unknown_dev/*"
unknown_test_regex = "/content/data/11-785-s23-hw2p2-verification/unknown_test/*"

# We load the images from known and unknown folders
unknown_dev_images = [Image.open(p) for p in tqdm(sorted(glob.glob(unknown_dev_regex)))]
unknown_test_images = [Image.open(p) for p in tqdm(sorted(glob.glob(unknown_test_regex)))]
known_images = [Image.open(p) for p in tqdm(sorted(glob.glob(known_regex)))]

# Why do you need only ToTensor() here?
transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()])

unknown_dev_images = torch.stack([transforms(x) for x in unknown_dev_images])
unknown_test_images = torch.stack([transforms(x) for x in unknown_test_images])
known_images  = torch.stack([transforms(y) for y in known_images ])
#Print your shapes here to understand what we have done

# You can use other similarity metrics like Euclidean Distance if you wish
similarity_metric = torch.nn.CosineSimilarity(dim= 1, eps= 1e-6) 

100%|██████████| 960/960 [00:00<00:00, 13946.97it/s]


In [ ]:
def eval_verification(unknown_images, known_images, model, similarity, batch_size= config['batch_size'], mode='val'): 

    unknown_feats, known_feats = [], []

    batch_bar = tqdm(total=len(unknown_images)//batch_size, dynamic_ncols=True, position=0, leave=False, desc=mode)
    model.eval()

    # We load the images as batches for memory optimization and avoiding CUDA OOM errors
    for i in range(0, unknown_images.shape[0], batch_size):
        unknown_batch = unknown_images[i:i+batch_size] # Slice a given portion upto batch_size
        
        with torch.no_grad():
            unknown_feat = model(unknown_batch.float().to(DEVICE), return_feats=True) #Get features from model         
        unknown_feats.append(unknown_feat)
        batch_bar.update()
    
    batch_bar.close()
    
    batch_bar = tqdm(total=len(known_images)//batch_size, dynamic_ncols=True, position=0, leave=False, desc=mode)
    
    for i in range(0, known_images.shape[0], batch_size):
        known_batch = known_images[i:i+batch_size] 
        with torch.no_grad():
              known_feat = model(known_batch.float().to(DEVICE), return_feats=True)
          
        known_feats.append(known_feat)
        batch_bar.update()

    batch_bar.close()

    # Concatenate all the batches
    unknown_feats = torch.cat(unknown_feats, dim=0)
    known_feats = torch.cat(known_feats, dim=0)

    similarity_values = torch.stack([similarity(unknown_feats, known_feature) for known_feature in known_feats])
    # Print the inner list comprehension in a separate cell - what is really happening?

    max_similarity_values, predictions = similarity_values.max(0) #Why are we doing an max here, where are the return values?
    max_similarity_values, predictions = max_similarity_values.cpu().numpy(), predictions.cpu().numpy()


    # Note that in unknown identities, there are identities without correspondence in known identities.
    # Therefore, these identities should be not similar to all the known identities, i.e. max similarity will be below a certain 
    # threshold compared with those identities with correspondence.

    # In early submission, you can ignore identities without correspondence, simply taking identity with max similarity value
    #pred_id_strings = [known_paths[i] for i in predictions] # Map argmax indices to identity strings
    
    # After early submission, remove the previous line and uncomment the following code 

    threshold = 0.3
    NO_CORRESPONDENCE_LABEL = 'n000000'
    pred_id_strings = []
    for idx, prediction in enumerate(predictions):
       if max_similarity_values[idx] < threshold: # why < ? Thank about what is your similarity metric
            pred_id_strings.append(NO_CORRESPONDENCE_LABEL)
       else:
            pred_id_strings.append(known_paths[prediction])
    
    if mode == 'val':
      true_ids = pd.read_csv('/content/data/11-785-s23-hw2p2-verification/verification_dev.csv')['label'].tolist()
      accuracy = accuracy_score(pred_id_strings, true_ids)
      print("Verification Accuracy = {}".format(accuracy))
    
    return pred_id_strings

In [ ]:
# verification eval
pred_id_strings = eval_verification(unknown_dev_images, known_images, model, similarity_metric, config['batch_size'], mode='val')
# verification test
pred_id_strings = eval_verification(unknown_test_images, known_images, model, similarity_metric, config['batch_size'], mode='test')

Verification Accuracy = 0.6472222222222223


In [ ]:
with open("verification_early_submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(pred_id_strings)):
        f.write("{},{}\n".format(i, pred_id_strings[i]))